# Proceso de ETL: User_Reviews

### Librerias necesarias

In [1]:
import pandas as pd 
import gzip
import ast
import warnings
warnings.filterwarnings("ignore")

## Extraction y primera exploración: 
Se lee el dataset que contiene las reviews de los usuarios y se guarda en un dataframe.

In [2]:
 #Leer el archivo JSON comprimido
def leer_jsoncomprimido():
    with gzip.open('../DatasetsSTEAM/user_reviews.json.gz','rt',encoding='utf-8')as file:
        return[ast.literal_eval(line.strip())
               for line in file]

#convertirlo a un dataframe
df_reviews=pd.DataFrame(leer_jsoncomprimido())

In [3]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


## Transformación: 
Se examina si existen valores nulos en las columnas.

In [4]:
# Verificamos si existen valores nulos en cada columna
df_reviews.isnull().sum().sort_values(ascending=False)

user_id     0
user_url    0
reviews     0
dtype: int64

### Verificación de duplicados

In [5]:
# Verificar registros duplicados en la columna 'user_id'
duplicados = df_reviews[df_reviews.duplicated(subset=['user_id'], keep=False)]
duplicados

,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


Se detectan 623 registros duplicados en la columna 'user_id'.<br>
Se procede a verificar si los review dentro de la columna anidada 'review' están duplicados o si únicamente se repite el 'user_id', ya que un usuario podria tener mas un comentario.

In [6]:
# Se revisa un usuario de ejemplo
user_id = 'yolofaceguy'
user_reviews = duplicados[duplicados['user_id'] == user_id]['reviews']
for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 25)

from the creaters of the walking dead, i present to you, the wolf among us. a twisted an unhappy place where crimes are made in the town of the fables. Ecperience one of the most mind-bending, jaw-dropping twists you will see in all of gaming history...well, some of gaming history. SPOILERS: i really liked how bigby turns into his true form to fight all the bloody marys... its just like when neo fought all the agent smiths in the matrix. what i didnt really like is when i chose to lock the crooked man up it glitches my game and i start from the near begining where bigby fought the crooked man's crew, exept the guy who runs a strip club wasnt there. so i was really confused. but anyway i really recommend this game if you want twists and ♥♥♥♥ed up scenes.
this game is awesome,this game is ♥♥♥♥ed up and this game is so sad and depressing. if you want these types of games i would strongly reccomend youto buy it. its worth your money
-------------------------
from the creaters of the walkin

Los 'reviews' son idénticos para cada registro, así que se eliminan los duplicados, conservando solo la primera ocurrencia de cada uno.

In [7]:
# Elimina filas duplicadas en df_reviews basadas en la columna 'user_id', manteniendo la primera ocurrencia de cada usuario
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')

### Se analiza la columna 'review' para entender su contenido.

In [8]:
# Se observa el tipo de dato que contiene 'review'
df_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

## Transformación de la columna 'reviews'
La columna 'reviews' contiene una lista anidada de diccionarios. El objetivo es crear una columna por cada diccionario para facilitar el registro individual de cada uno.

In [9]:
# Se transforma a columnas cada elemento de las listas
df_tmp = pd.json_normalize(df_reviews['reviews'])
df_tmp.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


En la transformación, se pierde la identificación del usuario ('user_id' y 'user_url'), pero se conserva la posición. Se recupera la trazabilidad del usuario al concatenarlo con el dataframe previo.

In [10]:
# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
df_tmp = pd.concat([df_reviews[['user_id', 'user_url']], df_tmp], axis=1)
df_tmp.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Los diccionarios por columnas se convierten en registros individuales, cada uno con el usuario que lo creó.

In [11]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
df_tmp = pd.melt(df_tmp, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_tmp.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


Se averigua los registros que tienen None en 'reviews' y luego se eliminan.

In [12]:
# Calcula la cantidad de valores nulos en la columna 'reviews'
df_tmp['reviews'].isnull().sum()

174023

In [13]:
# Se eliminan las filas con valor None
df_tmp = df_tmp.dropna()

En este punto ya es posible convertir cada diccionario en columna.

In [14]:
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = df_tmp['reviews'].apply(pd.Series, dtype='object')
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


En la etapa anterior, se pierden 'user_id' y 'user_url', por lo que se requiere volver a concatenar.

In [15]:
# Se une con el 'user_id' y 'user_url'
df_reviews = pd.concat([df_tmp[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se identifican valores faltantes en algunas columnas que podrían estar representados por espacios en lugar de nulos. Se procede a verificar esta situación.

In [16]:
#Accede al valor ubicado en la primera fila de la columna 'last_edited'
df_reviews['last_edited'][0]

''

Se reemplazan esas cadenas vacias con valores nulos.

In [17]:
# Reemplaza los valores de cadena vacía por None en el DataFrame df_reviews
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se analizan si las columnas tienen valores nulos que quedaron luego de desanidar la columna 'reviews'.

In [18]:
# Calcula la cantidad de valores nulos en cada columna del DataFrame df_reviews y los ordena de forma descendente
df_reviews.isnull().sum().sort_values(ascending=False)

last_edited    51499
funny          49498
review            30
user_id            0
user_url           0
posted             0
item_id            0
helpful            0
recommend          0
dtype: int64

Se detecta un alto porcentaje de datos faltantes (86-89%) en 'reviews_funny' y 'reviews_last_edited', por lo que se opta por eliminar estas columnas.

In [19]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited'
df_reviews = df_reviews.drop(columns=['funny', 'last_edited'])
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'helpful', 'recommend',
       'review'],
      dtype='object')

### Transformación de la columna 'posted'
Se necesita que la fecha donde se hizo el posteo de la review este en formato YYYY-MM-DD

In [20]:
# Convierte la columna 'posted' a un formato de fecha válido
df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format='Posted %B %d, %Y.', errors='coerce')
df_reviews.shape

(57397, 7)

In [21]:
# Filtra las filas cuya columna no tiene el formato especificado
df_reviews = df_reviews[df_reviews['posted'].notna()]
df_reviews.shape

(47626, 7)

### Columna 'review'
Esta columna tiene menos del 1% de valores nulos, por lo que se eliminan.

In [22]:
# Elimina filas del DataFrame df_reviews donde el valor en la columna 'review' es nulo (NaN)
df_reviews = df_reviews.dropna(subset=['review'])
df_reviews.shape

(47599, 7)

Eliminamos 'helpful' y 'user_url', ya que su información no sera necesario para utilizarlas en las funciones API mas adelante.

In [23]:
df_reviews = df_reviews.drop(['helpful','user_url'], axis=1)
df_reviews.columns

## Carga del dataset
Se guarda el dataframe transformado como steam_games.parquet

In [24]:
# Guarda el DataFrame df_reviews en formato Parquet en el archivo 'user_reviews.parquet' dentro de la carpeta 'Datasets'
df_reviews.to_parquet('../Datasets/user_reviews.parquet')